In [1]:
import or_gym
import numpy as np
env = or_gym.make('Knapsack-v0')

In [2]:
print("Max weight capacity:\t{}kg".format(env.max_weight))
print("Number of items:\t{}".format(env.N))

Max weight capacity:	200kg
Number of items:	200


In [3]:
env_config = {'N': 5,
             'max_weight': 15,
             'item_weights': np.array([1, 12, 2, 1, 4]),
             'item_values': np.array([2, 4, 2, 1, 10]),
             'mask': True}

env = or_gym.make('Knapsack-v0', env_config=env_config)
print("Max weight capacity:\t{}kg".format(env.max_weight))
print("Number of items:\t{}".format(env.N))

Max weight capacity:	15kg
Number of items:	5


In [4]:
env.state # weights -> values -> last number is total weight?

{'action_mask': array([1, 1, 1, 1, 1]),
 'avail_actions': array([1., 1., 1., 1., 1.]),
 'state': array([ 1, 12,  2,  1,  4,  2,  4,  2,  1, 10,  0])}

In [5]:
state, reward, done, _ = env.step(1)
print(state)

{'action_mask': array([1, 0, 1, 1, 0]), 'avail_actions': array([1., 1., 1., 1., 1.]), 'state': array([ 1, 12,  2,  1,  4,  2,  4,  2,  1, 10, 12])}


In [17]:
import ray
from ray.rllib import agents
from ray import tune
from ray.rllib.models import ModelCatalog
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models.tf.fcnet import FullyConnectedNetwork
from ray.rllib.utils import try_import_tf
from gym import spaces
from or_gym.utils import create_env
import tensorflow as tf 
#tf = try_import_tf()

<class 'function'>


In [19]:
class KP0ActionMaskModel(TFModelV2):
     
    def __init__(self, obs_space, action_space, num_outputs,
        model_config, name, true_obs_shape=(11,),
        action_embed_size=5, *args, **kwargs):
         
        super(KP0ActionMaskModel, self).__init__(obs_space,
            action_space, num_outputs, model_config, name, 
            *args, **kwargs)
         
        self.action_embed_model = FullyConnectedNetwork(
            spaces.Box(0, 1, shape=true_obs_shape), 
                action_space, action_embed_size,
            model_config, name + "_action_embedding")
        self.register_variables(self.action_embed_model.variables())
 
    def forward(self, input_dict, state, seq_lens):
        avail_actions = input_dict["obs"]["avail_actions"]
        action_mask = input_dict["obs"]["action_mask"]
        action_embedding, _ = self.action_embed_model({
            "obs": input_dict["obs"]["state"]})
        intent_vector = tf.expand_dims(action_embedding, 1)
        action_logits = tf.reduce_sum(avail_actions * intent_vector,
            axis=1)
        inf_mask = tf.maximum(tf.log(action_mask), tf.float32.min)
        return action_logits + inf_mask, state
 
    def value_function(self):
        return self.action_embed_model.value_function()

In [21]:
ModelCatalog.register_custom_model('kp_mask', KP0ActionMaskModel)

In [22]:
def register_env(env_name, env_config={}):
    env = create_env(env_name)
    tune.register_env(env_name, lambda env_name: env(env_name, env_config=env_config))
     
register_env('Knapsack-v0', env_config=env_config)

In [23]:
ray.init(ignore_reinit_error=True)
 
trainer_config = {
    "model": {
        "custom_model": "kp_mask"
        },
    "env_config": env_config
     }
trainer = agents.ppo.PPOTrainer(env='Knapsack-v0', config=trainer_config)

2021-05-28 18:20:32,699	INFO worker.py:651 -- Calling ray.init() again after it has already been called.
(pid=110888) WARNING:tensorflow:From /home_nfs/mimounis/anaconda3/lib/python3.7/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=110888) Instructions for updating:
(pid=110888) non-resource variables are not supported in the long term
(pid=110883) WARNING:tensorflow:From /home_nfs/mimounis/anaconda3/lib/python3.7/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=110883) Instructions for updating:
(pid=110883) non-resource variables are not supported in the long term
(pid=110888) WARNING:tensorflow:From /home_nfs/mimounis/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/resource_variable_ops.py:1666: ca

RayActorError: The actor died because of an error raised in its creation task, [36mray::RolloutWorker.__init__()[39m (pid=110883, ip=10.0.3.55)
  File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_executor
  File "/home_nfs/mimounis/anaconda3/lib/python3.7/site-packages/ray/_private/function_manager.py", line 556, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "/home_nfs/mimounis/anaconda3/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 514, in __init__
    self._build_policy_map(policy_dict, policy_config)
  File "/home_nfs/mimounis/anaconda3/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1155, in _build_policy_map
    policy_map[name] = cls(obs_space, act_space, merged_conf)
  File "/home_nfs/mimounis/anaconda3/lib/python3.7/site-packages/ray/rllib/policy/tf_policy_template.py", line 237, in __init__



(pid=110888)     self._build_policy_map(policy_dict, policy_config)
(pid=110888)   File "/home_nfs/mimounis/anaconda3/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1155, in _build_policy_map
(pid=110888)     policy_map[name] = cls(obs_space, act_space, merged_conf)
(pid=110888)   File "/home_nfs/mimounis/anaconda3/lib/python3.7/site-packages/ray/rllib/policy/tf_policy_template.py", line 237, in __init__
(pid=110888)     get_batch_divisibility_req=get_batch_divisibility_req,
(pid=110888)   File "/home_nfs/mimounis/anaconda3/lib/python3.7/site-packages/ray/rllib/policy/dynamic_tf_policy.py", line 294, in __init__
(pid=110888)     self._input_dict, self._state_inputs, self._seq_lens)
(pid=110888)   File "/home_nfs/mimounis/anaconda3/lib/python3.7/site-packages/ray/rllib/models/modelv2.py", line 234, in __call__
(pid=110888)     res = self.forward(restored, state or [], seq_lens)
(pid=110888)   File "<ipython-input-19-914fe3846d4a>", line 25, in forward
(pid=11

(pid=110884) WARNING:tensorflow:From /home_nfs/mimounis/anaconda3/lib/python3.7/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=110884) Instructions for updating:
(pid=110884) non-resource variables are not supported in the long term
(pid=110894) WARNING:tensorflow:From /home_nfs/mimounis/anaconda3/lib/python3.7/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=110894) Instructions for updating:
(pid=110894) non-resource variables are not supported in the long term
(pid=110884) WARNING:tensorflow:From /home_nfs/mimounis/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/resource_variable_ops.py:1666: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is

RayActorError: The actor died because of an error raised in its creation task, [36mray::RolloutWorker.__init__()[39m (pid=110884, ip=10.0.3.55)
  File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_executor
  File "/home_nfs/mimounis/anaconda3/lib/python3.7/site-packages/ray/_private/function_manager.py", line 556, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "/home_nfs/mimounis/anaconda3/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 514, in __init__
    self._build_policy_map(policy_dict, policy_config)
  File "/home_nfs/mimounis/anaconda3/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1155, in _build_policy_map
    policy_map[name] = cls(obs_space, act_space, merged_conf)
  File "/home_nfs/mimounis/anaconda3/lib/python3.7/site-packages/ray/rllib/policy/tf_policy_template.py", line 237, in __init__


In [15]:
env = trainer.env_creator('Knapsack-v0')
state = env.state
state['action_mask'][0] = 0

NameError: name 'trainer' is not defined